In [8]:
m, n = map(int, input().split(','))
if m!=n:
    print(f'Невозможно найти обратную матрицу: {m} != {n}')

3,3


In [6]:
matrix = []
for _ in range(m):
    k = list(map(int, input().split(',')))
    if len(k) > n:
        print("Неверная размерность матрицы")
        break
    matrix.append(k)

1,2,3
4,5,6
7,8,9


In [14]:
for i in zip(*matrix):
    print(i)

(1, 4, 7)
(2, 5, 8)
(3, 6, 9)


# 2 Задание

In [184]:
# Почистили руками ассоциативные повторяющиеся правила
edges=[
[1,2,300],
[1,4,150],
[1,7,220],
[2,1,100],
[2,5,520],
[2,10,110],
[3,4,340],
[4,1,150],
[4,3,340],
[5,2,520],
[7,1,220],
[9,10,230],
[10,2,110],
[10,9,230]
]
freqs = [
[1, 1234],
[2, 1505],
[3, 900],
[4, 2345],
[5, 378],
[6, 2998],
[7, 5421],
[8, 1323],
[9, 708],
[10, 1283]]

# Сортирую по частоте
freqs.sort(key=lambda x: x[1], reverse=True)

# Сортирую по релевантности
edges.sort(key=lambda x: x[2], reverse=True)

In [185]:
edges_dict = {'a_id':list(zip(*edges))[0], 'b_id':list(zip(*edges))[1], 'score':list(zip(*edges))[2]}
freqs_dict = {'id':list(zip(*freqs))[0], 'freq':list(zip(*freqs))[1]}

In [186]:
import pandas as pd

df_edges = pd.DataFrame(data=edges_dict)
df_freqs = pd.DataFrame(data=freqs_dict)


In [187]:
df_freqs

,id,freq
0,7,5421
1,6,2998
2,4,2345
3,2,1505
4,8,1323
5,10,1283
6,1,1234
7,3,900
8,9,708
9,5,378


In [188]:
df_edges

,a_id,b_id,score
0,2,5,520
1,5,2,520
2,3,4,340
3,4,3,340
4,1,2,300
5,9,10,230
6,10,9,230
7,1,7,220
8,7,1,220
9,1,4,150


In [189]:
# Сделаем фичу
# freq_2 - частота элемента b_id в edges
# score - score в edges
# Будем сортировать индексы в списке, составленного из ассоциативного списка, по следущему ключу:
# koef = freq_2*score
# Почему именно этот koef ? 
# Чем выше или ниже score, тем выше или ниже koef соответственно

# Чем выше или ниже частота объекта, который мы рекомендуем (freq_2), тем выше или ниже koef соответственно
# Это логично, так как чем большое популярность b_id (у объекта который мы рекомендуем), тем больше вероятность,
# того что пользователю понравится данный объект

# И получается следующие: чем больше ассоциативность и популярность товаров, тем больше шанс понравится потребителю
dict_freqs = {}
edge_koef = []
for freq in freqs:
    dict_freqs.update({freq[0]:freq[1]})
for edge in edges:
    edge_koef.append([edge[0], edge[1], (dict_freqs[edge[1]])*edge[2]])
edge_koef_dict = {'a_id':list(zip(*edge_koef))[0], 'b_id':list(zip(*edge_koef))[1], 'koef':list(zip(*edge_koef))[2]}
df_edge_koef = pd.DataFrame(data=edge_koef_dict).sort_values(by=["koef"], ascending=[False])
df_edge_koef

,a_id,b_id,koef
7,1,7,1192620
2,3,4,797300
1,5,2,782600
4,1,2,451500
9,1,4,351750
3,4,3,306000
5,9,10,295090
8,7,1,271480
0,2,5,196560
10,4,1,185100


In [346]:
def get_recomendation(basket, absent):
    # Алгоритм будет состоять из нескольких частей.
    
    # Сначала создаётся словарь из ассоциативных элементов, на его основе будет составляться ответ.
    
    # Пользователю куда интереснее объекты, которые хотя бы имеют ассоциативность.
    
    # Ассоциативность имеет большую значимость нежели популярность, 
    # так как между популярными элементами нет гарантированной связи,
    # поэтому в списке результатов первые будут стоять ассоциативные элементы, 
    # что гарантирует нам хоть какую-то связь.
    
    # Сортироваться элементы словаря будут по koef, его я расписал выше.
    # Если элементы повторяются, то их koef суммируются. 
    # Это логично, так как нам более двух раз попался один и тот же объект, 
    # значит выше вероятность того, что этот объект подойдет пользователю
    
    # Затем идёт проверка на повторения в баскете и в словаре.
    # Если в первых 4 элементах баскета есть какое либо значение из словаря,
    # то удаляем это значение из словаря.
    # Почему так ?
    # Если человек недавно пользовался объектам, то его не стоит рекомендовать ещё раз,
    # но если он когда-то им пользовался, то может его стоит порекомендовать ещё раз.
    
    # Формируется список из элементов отсортированных по частоте и при необходимости добавляется к ответу
    # Если вышло больше 4 элементов в ответе, то ответ обрезается
    
    # На выходе получаем список из 4 элементов
    
    size = 4
    
    absent_default = absent.copy()
    recs_edges = {}
    
    # Добавление ассоциативных элементов в словарь 
    for el in basket:
        for edge in edge_koef: 
            
            # Если нашли элемент в правилах
            if el == edge[0]:
                
                # Если нет элемента в запрете
                if edge[1] not in absent_default:
                    
                    
                    koef = 0
                    
                    # Если элемент уже присутствует в словаре, то складываем koef
                    if edge[1] in recs_edges.keys():
                        koef = edge[2]
                        recs_edges[edge[1]] += koef
                    else:
                        # Добавить элемент в словарь {b_id:koef}
                        recs_edges.update({edge[1]:edge[2]})
                    
                        
    # Сортировка правил по релевантности
    recs_edges = dict(sorted(recs_edges.items(), key=lambda item: item[1], reverse=True))
    
    """
    print("До удаления")
    print(f'recs_edges - {recs_edges}')
    print()
    """
    
    # Удаление из первых четырёх товаров
    for el in basket[:4]:
        if el in recs_edges.keys():
            #print(f"Удаление - {el}")
            recs_edges.pop(el)
            absent_default.append(el)
    
    # Формирование списка по частотам
    recs_freqs = [freq[0] for freq in freqs if freq[0] if freq[0] not in absent_default]
    
    """
    print("После удаления")
    print(f'recs_freqs - {recs_freqs}')
    print(f'recs_edges - {recs_edges.keys()}')
    print()
    """
    
    recs_lst = list(recs_edges.keys())
    i = len(recs_lst)
    
    # Добавление популярных товаров
    while i != size:
        recs_lst.append(recs_freqs.pop(0))
        i += 1
    
        
    # Срез списка
    if i > size:
        recs_lst = recs_lst[:4]
    
    """
    print(f'recs_freq - {recs_freqs}')
    print(f'recs_edges - {recs_lst}')
    """
    #print(basket, recs_lst, sep='\t')
    
    len_line = 50
    space = len_line - len(str(basket)) - len(str(recs_lst))
    stroka = f"{str(basket)} {str(recs_lst).rjust(space)}"
    print(stroka)
    #return recs_lst

In [347]:
absent = [8, 5]
basket = [2, 3, 7, 9, 1]
get_recomendation(basket, absent)

[2, 3, 7, 9, 1]          [4, 10, 1, 6]


In [352]:
import random

def get_absent():
    cnt = random.randint(1,4)
    absent = random.sample(list(zip(*freqs))[0], cnt)
    return absent

def get_basket():
    cnt = random.randint(1,5)
    basket = random.sample(list(zip(*freqs))[0], cnt)
    return basket

random.seed(424)
absent = get_absent()
baskets = []

for _ in range(5):
    baskets.append(get_basket())
print(f"absent - {absent}")
print(baskets)

absent - [4, 7, 1, 9]
[[4, 7], [1, 9], [2, 5], [1], [1, 8]]


In [351]:
print(f"absent\n{absent}\n")
print("basket", "recs", sep='\t')
for basket in baskets:
    get_recomendation(basket, absent)

absent
[4, 7, 1, 9]

basket	recs
[4, 7]                     [3, 6, 2, 8]
[1, 9]                   [2, 10, 6, 2]
[2, 5]                 [10, 6, 8, 10]
[1]                        [2, 6, 2, 8]
[1, 8]                     [2, 6, 2, 8]


In [338]:
basket = [5]
absent = [8, 7, 5]
get_recomendation(basket, absent)

In [339]:
matrix = [[1, 2],
         [3, 4]]
s = [[str(e) for e in row] for row in matrix]
print(s)
lens = [max(map(len, col)) for col in zip(*s)]
fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
table = [fmt.format(*row) for row in s]
print('\n'.join(table))

[['1', '2'], ['3', '4']]
1	2
3	4


In [280]:
basket = [8, 5, 3, 9]
rec = [4, 2, 10, 6]
len_line = 50

space = len_line - len(str(basket)) - len(str(rec))
stroka = f"{basket} {str(rec).rjust(space)}"
print(stroka)

[8, 5, 3, 9]             [4, 2, 10, 6]


In [140]:
list(zip(*freqs))

[(7, 6, 4, 2, 8, 10, 1, 3, 9, 5),
 (5421, 2998, 2345, 1505, 1323, 1283, 1234, 900, 708, 378)]

In [145]:
import random

def get_absent():
    cnt = random.randint(1,4)
    absent = random.sample(list(zip(*freqs))[0], cnt)
    return absent

def get_basket():
    cnt = random.randint(1,5)
    basket = random.sample(list(zip(*freqs))[0], cnt)
    return basket

random.seed(444)
absent = get_absent()
baskets = []

for _ in range(5):
    baskets.append(get_basket())

In [146]:
baskets

[[8, 5, 3, 9], [3, 6, 1], [2, 3, 7, 9, 1], [5], [10]]

In [9]:
lst = list("sbbbabasasfasffas")
lst.pop(0)
"".join(lst)

'bbbabasasfasffas'

In [432]:
class myDeque:
    def __init__(self, deque: str):
        self.deque = deque

    def add(self, element: str):
        """
        add element to start of deque
        :param element:
        """
        self.deque += element

    def pop(self):
        """
        :return: first deleted element
        """
        lst = list(self.deque)
        el = lst[0]
        lst.pop(0)
        self.deque = "".join(lst)
        return el

    def __eq__(self, other):
        """
        realization operator ==
        :param other: object myDeque
        :return: True or False
        """
        lst_1 = list(self.deque)
        lst_2 = list(other.deque)
        if len(lst_1) == len(lst_2):
            for i in range(len(lst_1)):
                el_1 = lst_1[i]
                el_2 = lst_2[i]
                if el_1 != el_2:
                    return False
            return True
        return False

    def __gt__(self, other):
        """
        realization operator >
        :param other: object myDeque
        :return: True or False
        """
        lst_1 = list(self.deque)
        lst_2 = list(other.deque)
        if len(lst_1) > len(lst_2):
            return True
        elif len(lst_1) < len(lst_2):
            return False
        else:
            for i in range(len(lst_1)):
                el_1 = lst_1[i]
                el_2 = lst_2[i]
                if el_1 > el_2:
                    return False
                elif el_1 < el_2:
                    return True
        return False

    def __lt__(self, other):
        """
        realization operator <
        :param other: object myDeque
        :return: True or False
        """
        if self == other:
            return False
        return (not self > other)


def parse_deques(a, b):
    """
    :param a: object myDeque
    :param b: object myDeque
    :return: prints the elements of two objects by priority
    """
    size_a = len(a.deque)
    size_b = len(b.deque)
    i = size_a + size_b
    while i != 0:
        while a > b:
            print(i, a.pop())
            i -= 1
        while b > a:
            print(i, b.pop())
            i -= 1
        if a == b and len(a.deque) != 0 and len(b.deque) != 0:
            print(i, a.pop())
            i -= 1

a = myDeque("AAB")
b = myDeque("BBB")

parse_deques(a, b)

In [438]:
a = myDeque("AGCA")
b = myDeque("AVB")

parse_deques(a, b)

7 A
6 A
5 G
4 C
3 V
2 A
1 B
